Kaggle notebook - https://www.kaggle.com/aagghh/is-astrology-right-testing-the-zodiac-claims

### Divorces Data exploration

In [ ]:
#import modules
import numpy as np 
import pandas as pd 
import seaborn as sns
from matplotlib import pyplot as plt
from scipy.stats import norm
from sklearn.preprocessing import StandardScaler
from scipy import stats
%matplotlib inline

#list all files under the input directory
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
#to see the data items
divorce_data = pd.read_csv('/kaggle/input/divorcemarriage-dataset-with-birth-dates/divorces_2000-2015_translated.csv')

divorce_data

In [ ]:
#dropping all coloumns except partners' birth dates
divorce_data.drop(divorce_data.columns.difference(['DOB_partner_man','DOB_partner_woman']), 1, inplace=True)

divorce_data.head

In [ ]:
#heatmap for missing values
sns.heatmap(divorce_data.isnull())

In [ ]:
#deleting all missing observations
divorce_data = divorce_data.dropna()

#cheking clean data
sns.heatmap(divorce_data.isnull())

divorce_data.head

In [ ]:
#cheking data types 
divorce_data.info()

In [ ]:
#set the right format for DOB columns 
divorce_data[['DOB_partner_man', 'DOB_partner_woman']] = divorce_data[['DOB_partner_man', 'DOB_partner_woman']].apply(pd.to_datetime)

#getting the day and month separately for both partners
divorce_data['day_partner_1'], divorce_data['month_partner_1'] = divorce_data['DOB_partner_man'].apply(lambda x: x.day), divorce_data['DOB_partner_man'].apply(lambda x: x.month)
divorce_data['day_partner_2'], divorce_data['month_partner_2'] = divorce_data['DOB_partner_woman'].apply(lambda x: x.day), divorce_data['DOB_partner_woman'].apply(lambda x: x.month)

#get rid of the DOB col as we don't need years
divorce_data = divorce_data.drop(['DOB_partner_man', 'DOB_partner_woman'], axis=1)

divorce_data.head(10)

In [ ]:
# function for the zodiac sign determination

def zodiac_sign(day, month): 
    
    if month == 12: 
        return 'Sagittarius' if (day < 22) else 'Capricorn'

    elif month == 1: 
        return 'Capricorn' if (day < 20) else 'Aquarius'

    elif month == 2: 
        return 'Aquarius' if (day < 19) else 'Pisces'

    elif month == 3: 
        return 'Pisces' if (day < 21) else 'Aries'

    elif month == 4: 
        return 'Aries' if (day < 20) else 'Taurus'

    elif month == 5: 
        return 'Taurus' if (day < 21) else 'Gemini'

    elif month == 6: 
        return 'Gemini' if (day < 21) else 'Cancer'

    elif month == 7: 
        return 'Cancer' if (day < 23) else 'Leo'

    elif month == 8: 
        return 'Leo' if (day < 23) else 'Virgo'

    elif month == 9: 
        return 'Virgo' if (day < 23) else 'Libra'

    elif month == 10: 
        return 'Libra' if (day < 23) else 'Scorpio'

    elif month == 11: 
        return 'Scorpio' if (day < 22) else 'Sagittarius'

In [ ]:
#creating additional cols for zodiac signs for both partners

divorce_data['Zod_sign_partner_1'] = divorce_data.apply(lambda x: zodiac_sign(x['day_partner_1'], x['month_partner_1']), axis=1)
divorce_data['Zod_sign_partner_2'] = divorce_data.apply(lambda x: zodiac_sign(x['day_partner_2'], x['month_partner_2']), axis=1)


In [ ]:
#checking our zodiac signs do make sense
divorce_data.head(10)

In [ ]:
#once they do make sense, we can delete all other cols exept zodiac signs
divorce_data.drop(divorce_data.columns.difference(['Zod_sign_partner_1','Zod_sign_partner_2']), 1, inplace=True)

divorce_data.head(10)

In [ ]:
#plotting a chart for a number of men zodiac signs across the data
plt.figure(figsize=(12,4))
plot = sns.countplot(x="Zod_sign_partner_1", data=divorce_data, palette="PuBu").set_title('Men Zodiac Signs')
plt.xlabel("")
plt.ylabel("count of zodiacs")

#making a one for women 
plt.figure(figsize=(12,4))
plot = sns.countplot(x="Zod_sign_partner_2", data=divorce_data, palette="gist_earth").set_title('Women Zodiac Signs')
plt.xlabel("")
plt.ylabel("count of zodiacs")

In [ ]:
#creating a matrix for the zodiac signs combinations (man+woman)
adjacency_matrix = pd.crosstab(divorce_data.Zod_sign_partner_1, divorce_data.Zod_sign_partner_2)
idx = adjacency_matrix.columns.union(adjacency_matrix.index)
adjacency_matrix = adjacency_matrix.reindex(index = idx, columns=idx, fill_value=0)
adjacency_matrix.head(12)

In [ ]:
#creating an additional col for the zodiac combinations
divorce_data['Zodiac_combinations'] = divorce_data['Zod_sign_partner_1'] + divorce_data['Zod_sign_partner_2']

#plotting a chart for combinations
plt.figure(figsize=(30,10))
plot = sns.countplot(x="Zodiac_combinations", data=divorce_data, palette="cividis_r").set_title('Zodiac Signs Combinations')
plt.xlabel("")
plt.ylabel("count of combinations")
plt.xticks(rotation=90)


In [ ]:
#see the histogram for the normal distribution
plt.figure(figsize=(12,6))
sns.distplot(divorce_data['Zodiac_combinations'].value_counts(), fit=norm);
plt.xlabel("")

In [ ]:
#upload the above compatibility matrix from a .csv 
comp_matrix = pd.read_csv('/kaggle/input/divorcemarriage-dataset-with-birth-dates/Comp_matrix.csv')

comp_matrix.head(10) #compatibility rate - %

In [ ]:
#making a scatter plot

plt.figure(figsize=(30,10))
x, comb = np.unique(comp_matrix['Zodiac_combination'], return_inverse=True)
plt.scatter(comb, comp_matrix['Compatibility_rate'])
plt.xticks(range(len(x)), x)
plt.xticks(rotation=90)
plt.show()

In [ ]:
#see the histogram for the normal distribution
plt.figure(figsize=(12,6))
sns.distplot(comp_matrix['Compatibility_rate'], fit=norm);
plt.xlabel("")

In [ ]:
#see the max/min, mean etc.
comp_matrix['Compatibility_rate'].describe()

In [ ]:
#setting the categories
compatibility_fit_labels = ['Bad_fit', 'Good_fit']

#creating a new col for the categories
comp_matrix['Compatibility'] = pd.qcut(comp_matrix['Compatibility_rate'], q= [0, .5, 1], labels=compatibility_fit_labels)

In [ ]:
#seeing how many values in each category
comp_matrix['Compatibility'].value_counts()

In [ ]:
#rename the col for future dataframes join
comp_matrix = comp_matrix.rename(columns={'Zodiac_combination': 'Zodiac_combinations'})

#joining the datframes
cols = ['Zodiac_combinations']
divorce_data = divorce_data.join(comp_matrix.set_index(cols), on=cols)

divorce_data

In [ ]:
# plotting a pie chart, to see how actually zodiac compatibility is distributed across the divorce dataset in percentage

labels = ['Good_fit', 'Bad_fit']
sizes = divorce_data['Compatibility'].value_counts()
#colors
colors = ['#ff9999','#66b3ff','#99ff99','#ffcc99']


fig1, ax1 = plt.subplots()
ax1.pie(sizes, labels=labels, autopct='%1.1f%%', colors = colors, shadow=True, startangle=90, pctdistance=0.85)

centre_circle = plt.Circle((0,0),0.70,fc='white')
fig = plt.gcf()
fig.gca().add_artist(centre_circle)
fig.set_size_inches(6,6)
ax1.axis('equal')  # Equal aspect ratio ensures that pie is drawn as a circle.
plt.tight_layout()
plt.title('Zodiac Compatibility: Divorce Data (percentage)')
plt.show()

In [ ]:
#plotting a chart to see how zodiac compatibility is distributed across the divorce dataset in actual numbers
plt.figure(figsize=(12,4))
plot = sns.countplot(x="Compatibility", data=divorce_data, facecolor=(0, 0, 0, 0),linewidth=5,edgecolor=sns.color_palette("dark", 3)).set_title('Zodiac Compatibility: Divorce Data (actual numbers)')
plt.xlabel("")
plt.ylabel("")

In [ ]:
#see the number 
divorce_data['Compatibility'].value_counts()